# Unit Checked Arithmetic via Unitful.jl
### Chris Rackauckas

Units and dimensional analysis are standard tools across the sciences for checking the correctness of your equation. However, most ODE solvers only allow for the equation to be in dimensionless form, leaving it up to the user to both convert the equation to a dimensionless form, punch in the equations, and hopefully not make an error along the way.

DifferentialEquations.jl allows for one to use Unitful.jl to have unit-checked arithmetic natively in the solvers. Given the dispatch implementation of the Unitful, this has little overhead.

## Using Unitful

To use Unitful, you need to have the package installed. Then you can add units to your variables. For example:

In [ ]:
using Unitful
t = 1.0u"s"

Notice that `t` is a variable with units in seconds. If we make another value with seconds, they can add

In [ ]:
t2 = 1.02u"s"
t+t2

and they can multiply:

In [ ]:
t*t2

You can even do rational roots:

In [ ]:
sqrt(t)

Many operations work. These operations will check to make sure units are correct, and will throw an error for incorrect operations:

In [ ]:
t + sqrt(t)

## Using Unitful with DifferentialEquations.jl

Just like with other number systems, you can choose the units for your numbers by simply specifying the units of the initial condition and the timestep. For example, to solve the linear ODE where the variable has units of Newton's and `t` is in Seconds, we would use:

In [ ]:
using DifferentialEquations
f = (y,p,t) -> 0.5*y
u0 = 1.5u"N"
prob = ODEProblem(f,u0,(0.0u"s",1.0u"s"))
sol = solve(prob,Tsit5())

Notice that we recieved a unit mismatch error. This is correctly so! Remember that for an ODE:

$$\frac{dy}{dt} = f(t,y)$$

we must have that `f` is a rate, i.e. `f` is a change in `y` per unit time. So we need to fix the units of `f` in our example to be `N/s`. Notice that we then do not receive an error if we do the following:

In [ ]:
f = (y,p,t) -> 0.5*y/3.0u"s"
prob = ODEProblem(f,u0,(0.0u"s",1.0u"s"))
sol = solve(prob,Tsit5())

This gives a a normal solution object. Notice that the values are all with the correct units:

In [ ]:
print(sol[:])

We can plot the solution by removing the units:

In [ ]:
using Plots
gr()
plot(ustrip(sol.t),ustrip(sol[:]),lw=3)